<a href="https://colab.research.google.com/github/sabrinabenb/Graph-Neural-Network-GNN/blob/main/GAT_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git


2.4.0+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 20.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

dataset = Planetoid(root='data/Planetoid', name='Cora', transform=NormalizeFeatures())

print()
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]
data

Processing...



Dataset: Cora():
Number of graphs: 1
Number of features: 1433
Number of classes: 7


Done!


Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [ ]:
import argparse
import os.path as osp
import time

import torch
import torch.nn.functional as F

import torch_geometric
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.logging import init_wandb, log
from torch_geometric.nn import GATConv,GATv2Conv



if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch_geometric.is_xpu_available():
    device = torch.device('xpu')
else:
    device = torch.device('cpu')

init_wandb(name=f'GAT-{dataset}', heads=8, epochs=200,
           hidden_channels=16, lr=0.005, device=device)


data = dataset[0].to(device)


class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads):
        super().__init__()
        self.conv1 = GATv2Conv(in_channels, hidden_channels, heads, dropout=0.6)
        # On the Pubmed dataset, use `heads` output heads in `conv2`.
        self.conv2 = GATv2Conv(hidden_channels * heads, out_channels, heads=1,
                             concat=False, dropout=0.6)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return x


model = GAT(dataset.num_features, 16, dataset.num_classes,
            8).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)


def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return float(loss)


@torch.no_grad()
def test():
    model.eval()
    pred = model(data.x, data.edge_index).argmax(dim=-1)

    accs = []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        accs.append(int((pred[mask] == data.y[mask]).sum()) / int(mask.sum()))
    return accs


times = []
best_val_acc = final_test_acc = 0
for epoch in range(1, 200 + 1):
    start = time.time()
    loss = train()
    train_acc, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)
    times.append(time.time() - start)
print(f"Median time per epoch: {torch.tensor(times).median():.4f}s")

Epoch: 001, Loss: 1.9502, Train: 0.1571, Val: 0.3100, Test: 0.3150
Epoch: 002, Loss: 1.9403, Train: 0.1857, Val: 0.1340, Test: 0.3150
Epoch: 003, Loss: 1.9265, Train: 0.2214, Val: 0.1380, Test: 0.3150
Epoch: 004, Loss: 1.9212, Train: 0.2857, Val: 0.2580, Test: 0.3150
Epoch: 005, Loss: 1.9101, Train: 0.3214, Val: 0.2580, Test: 0.3150
Epoch: 006, Loss: 1.9007, Train: 0.5143, Val: 0.3280, Test: 0.3410
Epoch: 007, Loss: 1.8977, Train: 0.6286, Val: 0.4220, Test: 0.4360
Epoch: 008, Loss: 1.8815, Train: 0.8500, Val: 0.6640, Test: 0.6730
Epoch: 009, Loss: 1.8784, Train: 0.9286, Val: 0.7780, Test: 0.7870
Epoch: 010, Loss: 1.8593, Train: 0.9000, Val: 0.7280, Test: 0.7870
Epoch: 011, Loss: 1.8577, Train: 0.8286, Val: 0.6380, Test: 0.7870
Epoch: 012, Loss: 1.8456, Train: 0.7857, Val: 0.5780, Test: 0.7870
Epoch: 013, Loss: 1.8221, Train: 0.7786, Val: 0.5340, Test: 0.7870
Epoch: 014, Loss: 1.8254, Train: 0.7571, Val: 0.4460, Test: 0.7870
Epoch: 015, Loss: 1.8106, Train: 0.8071, Val: 0.4640, Test: 0.